# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [98]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [99]:
# Loading csv file from WeatherPy
cities_df = pd.read_csv("../WeatherPy/output_data/cities.csv",index_col=0)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Nantucket,41.28,-70.10,75.20,83,1,9.17,US,1596287482
1,Port Alfred,-33.59,26.89,65.80,60,0,8.90,ZA,1596287706
2,Badajoz,38.67,-6.17,96.01,14,0,5.01,ES,1596287706
3,Boshnyakovo,49.63,142.17,58.48,93,100,9.13,RU,1596287706
4,Aljezur,37.32,-8.80,84.99,44,3,13.73,PT,1596287707


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [100]:
# Configuring gmaps for heatmap creation
gmaps.configure(api_key=g_key)

In [105]:
# Setting locations by lat/long and intensity by humidity
locations = cities_df[['Lat','Lng']].astype(float)
humidity = cities_df['Humidity'].astype(float)

In [106]:
# Creating heat map of humidity
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [107]:
# Setting conditions for ideal vacation spots to get our tan on
hotel_df = cities_df.where((cities_df['Max Temp']>75) &
                                 (cities_df['Max Temp']<90) &
                                 (cities_df['Wind Speed']<10) &
                                 (cities_df['Cloudiness']==0) &
                                 (cities_df['Humidity']<80))
hotel_df=hotel_df.dropna()
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,Rājbirāj,26.53,86.75,89.13,60.0,0.0,2.15,NP,1.596287e+09
174,Richards Bay,-28.78,32.04,80.38,34.0,0.0,8.12,ZA,1.596288e+09
213,Kumul,42.80,93.45,88.34,11.0,0.0,1.86,CN,1.596288e+09
243,Nesoddtangen,59.87,10.66,75.99,55.0,0.0,4.00,NO,1.596288e+09
270,Guigue,10.09,-67.78,77.40,76.0,0.0,3.78,VE,1.596288e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [108]:
# Creating new column in the df to store the name of the hotels in our vacation spots
hotel_df['Hotel Name']=""

In [109]:
# Setting up the url and data pull from google maps for hotel names
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Creating search radius and target search type
target_radius=5000
target_search='hotel'

# Setting search parameters
params = {"keyword": target_search,
          "radius": target_radius,
          "key": g_key,
          }

# iterating through the df to get the nearest hotels, 5000 m or less
for index, row in hotel_df.iterrows():
    
    city = row['City']
    lat = row['Lat']
    lng = row['Lng']
    params['location']=f'{lat},{lng}'
    
    # making API request
    print(f'Retrieving Results for {city}.'')
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f'Closest hotel to {city} is {results[0]['name']}.')
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Rājbirāj.
Closest hotel to Rājbirāj is Deluxe Star Hotel 3.
------------
Retrieving Results for Richards Bay.
Closest hotel to Richards Bay is BON Hotel Waterfront Richards Bay.
------------
Retrieving Results for Kumul.
Closest hotel to Kumul is 7 Days Inn.
------------
Retrieving Results for Nesoddtangen.
Closest hotel to Nesoddtangen is Grand Hotel Oslo.
------------
Retrieving Results for Guigue.
Closest hotel to Guigue is Motel paraiso.
------------
Retrieving Results for Fukuechō.
Closest hotel to Fukuechō is GOTO TSUBAKI HOTEL.
------------
Retrieving Results for Saldanha.
Closest hotel to Saldanha is Protea Hotel by Marriott Saldanha Bay.
------------
Retrieving Results for Malbork.
Closest hotel to Malbork is Hotel Grot.
------------
Retrieving Results for Luena.
Closest hotel to Luena is Hotel Kandamba.
------------
Retrieving Results for Derzhavīnsk.
Closest hotel to Derzhavīnsk is Gostinitsa Galaktika.
------------
Retrieving Results for Louis Trichar

In [110]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [111]:
# Creating marker layer for hotel locations
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)


# Displaying the figure
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))